In [1]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
import cv2
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import wandb

/Users/mitanshk/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.login(key='288db8f5afccc57f269ae50ac116d15047b1b705')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mitanshkayathwal (mmavericks). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/mitanshk/.netrc


True

In [3]:
source_directory = './double_mnist_seed_123_image_size_64_64/train/'

X_train = []
y_train = []

for root, dirs, files in os.walk(source_directory):
    for file in files:
        if file[0]==".":
            continue
        subdirectory_name = os.path.basename(root)
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        X_train.append(image)
        one_hot_outp = [0] * 10
        one_hot_outp[int(subdirectory_name[0])]=1
        one_hot_outp[int(subdirectory_name[1])]=1
        y_train.append(one_hot_outp)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [4]:
source_directory = './double_mnist_seed_123_image_size_64_64/val/'

X_val = []
y_val = []

for root, dirs, files in os.walk(source_directory):
    for file in files:
        if file[0]==".":
            continue
        subdirectory_name = os.path.basename(root)
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        X_val.append(image)
        one_hot_outp = [0] * 10
        one_hot_outp[int(subdirectory_name[0])]=1
        one_hot_outp[int(subdirectory_name[1])]=1
        y_val.append(one_hot_outp)

X_val = np.array(X_val)
y_val = np.array(y_val)

In [5]:
source_directory = './double_mnist_seed_123_image_size_64_64/test/'

X_test = []
y_test = []

for root, dirs, files in os.walk(source_directory):
    for file in files:
        if file[0]==".":
            continue
        subdirectory_name = os.path.basename(root)
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        X_test.append(image)
        one_hot_outp = [0] * 10
        one_hot_outp[int(subdirectory_name[0])]=1
        one_hot_outp[int(subdirectory_name[1])]=1
        y_test.append(one_hot_outp)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
class MLPDoubleMNIST(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, activation_type='sigmoid'):
        super(MLPDoubleMNIST, self).__init__()
        self.layers = nn.ModuleList()
        self.activation_type = activation_type
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.layers.append(self.activation(activation_type))
        
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.layers.append(self.activation(activation_type))
    
        self.layers.append(nn.Linear(hidden_sizes[-1], num_classes))
        self.layers.append(nn.Sigmoid())

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for layer in self.layers:
            x = layer(x)
        return x

    def activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'tanh':
            return nn.Tanh()
        elif activation == 'sigmoid':
            return nn.Sigmoid()

    def train(self, X, y, optimizer):
        outputs = self.forward(X)
        loss = nn.CrossEntropyLoss()(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        return loss.item()

    def predict(self, X):
        outputs = self.forward(X)
        outputs = outputs
        outputs = (outputs > 0.5).float()
        return outputs
    
    def evaluate_accuracy(self, X, y):
        predicted = self.predict(X)
        correct = (predicted == y).all(dim=1).sum()
        accuracy = correct.item() / len(y)
        return accuracy

In [10]:
train_dataset = TensorDataset(torch.from_numpy(X_train).float().unsqueeze(1), torch.from_numpy(y_train).float())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(torch.from_numpy(X_val).float().unsqueeze(1), torch.from_numpy(y_val).float())
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)

input_size = 64**2
hidden_sizes = [128, 64] 
num_classes = 10
batch_size = 64
model = MLPDoubleMNIST(input_size, hidden_sizes, num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    train_loss = 0.0
    train_accuracy = 0
    val_loss = 0
    val_accuracy = 0
    total_train = 0
    total_val = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        train_loss += loss
        total_train = total_train + 1
        train_accuracy += model.evaluate_accuracy(inputs, labels)

    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        val_loss += loss
        total_val = total_val + 1
        val_accuracy += model.evaluate_accuracy(inputs, labels)
    
    train_accuracy = 100*train_accuracy/total_train
    val_accuracy = 100*val_accuracy/total_val

    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1, Train Loss: 4.270215692996978, Train Accuracy: 0.97%
Epoch 1, Validation Loss: 4.154553555488587, Validation Accuracy: 2.98%
Epoch 2, Train Loss: 4.234031669855118, Train Accuracy: 1.57%
Epoch 2, Validation Loss: 4.114993722915649, Validation Accuracy: 3.19%
Epoch 3, Train Loss: 4.196339564323425, Train Accuracy: 2.67%
Epoch 3, Validation Loss: 4.068674428939819, Validation Accuracy: 4.23%
Epoch 4, Train Loss: 4.158682403087616, Train Accuracy: 2.65%
Epoch 4, Validation Loss: 4.028487662315369, Validation Accuracy: 4.96%
Epoch 5, Train Loss: 4.136294132471084, Train Accuracy: 2.89%
Epoch 5, Validation Loss: 3.992941209793091, Validation Accuracy: 6.08%
Epoch 6, Train Loss: 4.113805425405502, Train Accuracy: 2.99%
Epoch 6, Validation Loss: 3.977691025733948, Validation Accuracy: 5.55%
Epoch 7, Train Loss: 4.096072979927063, Train Accuracy: 3.04%
Epoch 7, Validation Loss: 3.9687238235473634, Validation Accuracy: 5.59%
Epoch 8, Train Loss: 4.085781814575196, Train Accuracy: 3.21%

In [91]:
hidden_layers = [[10], [128,64], [8,8], [10, 10], [128, 64, 32]]

for layers in hidden_layers:
    run = wandb.init(
                        project="SMAI_As3_Q5_1", entity="mitanshkayathwal"
                    )
    input_size = 64**2
    num_classes = 10
    batch_size = 64
    model = MLPDoubleMNIST(input_size, layers, num_classes)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10

    for epoch in range(num_epochs):
        train_loss = 0.0
        train_accuracy = 0
        total_train = 0

        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            loss = model.train(inputs, labels, optimizer)
            train_loss += loss
            total_train = total_train + 1
            train_accuracy += model.evaluate_accuracy(inputs, labels)
    
    val_accuracy = 0
    val_loss = 0
    total_val = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        val_loss += loss
        total_val = total_val + 1
        val_accuracy += model.evaluate_accuracy(inputs, labels)

    val_accuracy = (val_accuracy/total_val) * 100

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    train_accuracy /=total_train
    val_accuracy /=total_val

    wandb.log({         'Hidden Layers': layers,
                        'Validation Accuracy': val_accuracy,
                        'Validation Loss': val_loss,
                        'Train Loss': train_loss,
                        'Training Accuracy': train_accuracy,
            })

wandb: Currently logged in as: mitanshkayathwal. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,4.21137
Training Accuracy,0.00469
Validation Accuracy,0.00658
Validation Loss,4.15625


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,4.07068
Training Accuracy,0.03122
Validation Accuracy,0.01703
Validation Loss,4.01656


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,4.24778
Training Accuracy,0.02252
Validation Accuracy,0.01628
Validation Loss,4.20591


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,4.25099
Training Accuracy,0.01194
Validation Accuracy,0.01848
Validation Loss,4.19889


# Best Hyperparameters found using WandB:

Hidden Sizes: [10, 10]

Training Accuracy: 1.19%

Validation Accuracy: 1.8%

Training Loss: 0.03413

Validation Loss: 4.054

In [95]:
input_size = 64**2
num_classes = 10
batch_size = 64
model = MLPDoubleMNIST(input_size, [10, 10], num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        model.train(inputs, labels, optimizer)

accuracy = 100 * model.evaluate_accuracy(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
print("Accuracy on Test Dataset is: " + str(accuracy) + "%")

Accuracy on Test Dataset is: 0.5700000000000001%


In [6]:
class CNNModel(nn.Module):
    def __init__(self, kernel_sz=3, dropout_rate=0.25):
        super(CNNModel, self).__init__()  
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_sz, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_sz, padding=1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Updated the input size of fc1
        self.fc2 = nn.Linear(128, 10) 
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.activation(x)
        return x

In [60]:
model = CNNModel()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct_train = 0
    correct_val = 0
    total_val = 0
    val_loss = 0
    total_train = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model.forward(images)
        binary_predictions = (outputs > 0.5).float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train += labels.size(0)
        train_loss += loss.item()
        correct_train += (binary_predictions == labels).all(dim=1).sum().item()

    for images, labels in val_loader:
        outputs = model.forward(images)
        binary_predictions = (outputs > 0.5).float()
        loss = criterion(outputs, labels)
        total_val += labels.size(0)
        val_loss += loss.item()
        correct_val += (binary_predictions == labels).all(dim=1).sum().item()
    
    train_accuracy = 100 * correct_train/total_train
    val_accuracy = 100 * correct_val/total_val
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader)}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Epoch {epoch + 1}, Validation Loss: {train_loss / len(train_loader)}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1, Train Loss: 0.36573336184024813, Train Accuracy: 32.09%
Epoch 1, Validation Loss: 0.36573336184024813, Validation Accuracy: 15.97%
Epoch 2, Train Loss: 0.12150938247516752, Train Accuracy: 65.23%
Epoch 2, Validation Loss: 0.12150938247516752, Validation Accuracy: 25.29%
Epoch 3, Train Loss: 0.08455918423458934, Train Accuracy: 75.60%
Epoch 3, Validation Loss: 0.08455918423458934, Validation Accuracy: 27.04%
Epoch 4, Train Loss: 0.06675204199366272, Train Accuracy: 80.71%
Epoch 4, Validation Loss: 0.06675204199366272, Validation Accuracy: 29.90%
Epoch 5, Train Loss: 0.05776210439018905, Train Accuracy: 83.39%
Epoch 5, Validation Loss: 0.05776210439018905, Validation Accuracy: 33.36%
Epoch 6, Train Loss: 0.04999620796740055, Train Accuracy: 85.33%
Epoch 6, Validation Loss: 0.04999620796740055, Validation Accuracy: 32.31%
Epoch 7, Train Loss: 0.044873074002563955, Train Accuracy: 86.87%
Epoch 7, Validation Loss: 0.044873074002563955, Validation Accuracy: 35.23%
Epoch 8, Train Los

In [22]:
learning_rates = [0.001, 0.008]
kernel_sizes = [3, 5]
dropout_rates = [0, 0.25]

for lr in learning_rates:
    for kernel_sz in kernel_sizes:
        for dropout_rate in dropout_rates:
            run = wandb.init(
                        project="SMAI_As3_Q5_2", entity="mitanshkayathwal"
                    )
            
            model = CNNModel(kernel_sz, dropout_rate)

            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr)
            num_epochs = 10
            
            for epoch in range(num_epochs):
                model.train()
                train_loss = 0
                correct_train = 0
                correct_val = 0
                total_val = 0
                val_loss = 0
                total_train = 0

                for images, labels in train_loader:
                    optimizer.zero_grad()
                    outputs = model.forward(images)
                    binary_predictions = (outputs > 0.5).float()
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    total_train += labels.size(0)
                    train_loss += loss.item()
                    correct_train += (binary_predictions == labels).all(dim=1).sum().item()

                for images, labels in val_loader:
                    outputs = model.forward(images)
                    binary_predictions = (outputs > 0.5).float()
                    loss = criterion(outputs, labels)
                    total_val += labels.size(0)
                    val_loss += loss.item()
                    correct_val += (binary_predictions == labels).all(dim=1).sum().item()
                
                train_accuracy = 100 * correct_train/total_train
                val_accuracy = 100 * correct_val/total_val

                val_loss /= total_val
                train_loss /= total_train

                wandb.log({"Learning Rate": lr,
                           "Kernel Size": kernel_sz,
                           "Dropout Rate": dropout_rate,
                           "Train Accuracy": train_accuracy,
                           "Validation Accuracy": val_accuracy,
                           "Train Loss": train_loss,
                           "Validation Loss": val_loss
                           })

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Dropout Rate,▁▁▁▁▁▁▁▁▁▁
Kernel Size,▁▁▁▁▁▁▁▁▁▁
Learning Rate,▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▁▁▁▁▁▁
Train Loss,▁█████████
Validation Accuracy,▁▁▁▁▁▁▁▁▁▁
Validation Loss,▁▁▁▁▁▁▁▁▁▁
Dropout Rate,0
Kernel Size,3
Learning Rate,0.008
Train Accuracy,1.5625


# Best Hyperparameters found using WandB:

Kernel Size: 3

Learning Rate: 0.001

Training Accuracy: 95.74%

Validation Accuracy: 43.36%

Training Loss: 0.0002035

Validation Loss: 0.007916


In [11]:
model = CNNModel(3, 0.25)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [12]:
output = model.forward(torch.from_numpy(X_test).float().unsqueeze(1))
binary_predictions = (output > 0.5).float()
accuracy = 100*(binary_predictions == torch.from_numpy(y_test).float()).all(dim=1).sum().item()/len(y_test)
print(accuracy)

39.355


In [157]:
binary_predictions = (output > 0.5).float()
accuracy = 100*(binary_predictions == torch.from_numpy(y_test).float()).all(dim=1).sum().item()/len(y_test)
print(accuracy)

42.075


In [3]:
data = np.load("permuted_mnist.npz")
permX_train = data["train_images"]
permy_train = data["train_labels"]
permX_test = data["test_images"]
permy_test = data["test_labels"]
permX_train, permX_val, permy_train, permy_val = train_test_split(permX_train, permy_train, random_state=42,test_size=0.2)

In [4]:
# Write MLP Code using inbuilt functions

class MLPPermutedMNIST(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, activation_type='sigmoid'):
        super(MLPPermutedMNIST, self).__init__()
        self.layers = nn.ModuleList()
        self.activation_type = activation_type
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.layers.append(self.activation(activation_type))
        
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.layers.append(self.activation(activation_type))
    
        self.layers.append(nn.Linear(hidden_sizes[-1], num_classes))
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)

    def activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'tanh':
            return nn.Tanh()
        elif activation == 'sigmoid':
            return nn.Sigmoid()

    def train(self, X, y, optimizer):
        outputs = self.forward(X)
        loss = nn.CrossEntropyLoss()(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        return loss.item()

    def predict(self, X):
        outputs = self.forward(X)
        outputs = outputs.detach().numpy()
        outputs = np.argmax(outputs,axis=1)
        return outputs
    
    def evaluate_accuracy(self, X, y):
        predicted = self.predict(X)
        correct = (predicted == y).sum()
        accuracy = correct.item() / len(y)
        return accuracy

In [5]:
perm_train_dataset = TensorDataset(torch.from_numpy(permX_train).float().unsqueeze(1), torch.from_numpy(permy_train).long())
perm_train_loader = torch.utils.data.DataLoader(perm_train_dataset, batch_size=64, shuffle=True)

perm_val_dataset = TensorDataset(torch.from_numpy(permX_val).float().unsqueeze(1), torch.from_numpy(permy_val).long())
perm_val_loader = torch.utils.data.DataLoader(perm_val_dataset, batch_size=64, shuffle=True)

perm_test_dataset = TensorDataset(torch.from_numpy(permX_test).float().unsqueeze(1), torch.from_numpy(permy_test).long())
perm_test_loader = torch.utils.data.DataLoader(perm_test_dataset, batch_size=64, shuffle=True)

input_size = 28**2
hidden_sizes = [128, 128, 128] 
num_classes = 10
batch_size = 64
model = MLPPermutedMNIST(input_size, hidden_sizes, num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25

for epoch in range(num_epochs):
    train_loss = 0.0
    train_accuracy = 0
    val_accuracy = 0
    val_loss = 0
    for i, data in enumerate(perm_train_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        train_loss += loss
        train_accuracy += model.evaluate_accuracy(inputs, labels.numpy())
    
    for i, data in enumerate(perm_val_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        val_loss += loss
        val_accuracy += model.evaluate_accuracy(inputs, labels.numpy())

    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / len(perm_train_loader)}, Train Accuracy: {100*train_accuracy/len(perm_train_loader):.2f}%")
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(perm_val_loader)}, Validation Accuracy: {100*val_accuracy/len(perm_val_loader):.2f}%")

Epoch 1, Train Loss: 0.8968610704739889, Train Accuracy: 74.96%
Epoch 1, Validation Loss: 0.4840844971068362, Validation Accuracy: 86.77%
Epoch 2, Train Loss: 0.39967430410782495, Train Accuracy: 88.77%
Epoch 2, Validation Loss: 0.34695711002704943, Validation Accuracy: 90.55%
Epoch 3, Train Loss: 0.3443723158836365, Train Accuracy: 90.14%
Epoch 3, Validation Loss: 0.3410698729468153, Validation Accuracy: 89.83%
Epoch 4, Train Loss: 0.3272150169114272, Train Accuracy: 90.58%
Epoch 4, Validation Loss: 0.30884806411538984, Validation Accuracy: 91.02%
Epoch 5, Train Loss: 0.3080934458176295, Train Accuracy: 91.01%
Epoch 5, Validation Loss: 0.268357076662335, Validation Accuracy: 92.27%
Epoch 6, Train Loss: 0.28879182343681653, Train Accuracy: 91.40%
Epoch 6, Validation Loss: 0.2598650990093642, Validation Accuracy: 92.54%
Epoch 7, Train Loss: 0.2752088372906049, Train Accuracy: 92.05%
Epoch 7, Validation Loss: 0.25193137102859453, Validation Accuracy: 92.47%
Epoch 8, Train Loss: 0.2499197

In [10]:
hidden_layers = [[10], [128,64], [8,8], [10, 10], [128, 64, 32]]

best_accuracy = 0
best_params = None

for layers in hidden_layers:
    run = wandb.init(
                        project="SMAI_As3_Q5_3", entity="mitanshkayathwal"
                    )
    input_size = 28**2
    num_classes = 10
    batch_size = 64
    model = MLPPermutedMNIST(input_size, layers, num_classes)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 25

    for epoch in range(num_epochs):
        train_loss = 0.0
        train_accuracy = 0
        val_accuracy = 0
        val_loss = 0
        for i, data in enumerate(perm_train_loader, 0):
            inputs, labels = data
            loss = model.train(inputs, labels, optimizer)
            train_loss += loss
            train_accuracy += model.evaluate_accuracy(inputs, labels.numpy())
        
    for i, data in enumerate(perm_val_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)
        val_loss += loss
        val_accuracy += model.evaluate_accuracy(inputs, labels.numpy())

    val_accuracy = val_accuracy / len(perm_val_loader)
    if(best_accuracy<val_accuracy):
        best_accuracy=val_accuracy
        best_params = layers

    train_loss /= len(perm_train_loader)
    val_loss /= len(perm_val_loader)

    train_accuracy /= len(perm_train_loader)

    wandb.log({         'Hidden Layers': layers,
                        'Validation Accuracy': val_accuracy,
                        'Validation Loss': val_loss,
                        'Train Loss': train_loss,
                        'Training Accuracy': train_accuracy,
            })

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,0.45125
Training Accuracy,0.86665
Validation Accuracy,0.87134
Validation Loss,0.4549


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,0.17558
Training Accuracy,0.94817
Validation Accuracy,0.94847
Validation Loss,0.17923


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,0.64163
Training Accuracy,0.8194
Validation Accuracy,0.80627
Validation Loss,0.67417


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Loss,▁
Training Accuracy,▁
Validation Accuracy,▁
Validation Loss,▁
Train Loss,0.43493
Training Accuracy,0.88306
Validation Accuracy,0.8814
Validation Loss,0.45052


# Best Hyperparameters found using WandB:

Hidden Sizes: [128, 64]

Training Accuracy: 94.82%

Validation Accuracy: 94.85%

Training Loss: 0.1756

Validation Loss: 0.1792

In [11]:
test_accuracy = 0

input_size = 28**2
num_classes = 10
batch_size = 64
model = MLPPermutedMNIST(input_size, [128, 64], num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25

for epoch in range(num_epochs):
    for i, data in enumerate(perm_train_loader, 0):
        inputs, labels = data
        loss = model.train(inputs, labels, optimizer)

for i, data in enumerate(perm_test_loader, 0):
        inputs, labels = data
        test_accuracy += model.evaluate_accuracy(inputs, labels.numpy())

test_accuracy = test_accuracy / len(perm_test_loader)

print("Test Acuracy on MLP for Permuted MNIST is: " + str(test_accuracy*100))

Test Acuracy on MLP for Permuted MNIST is: 94.29737261146497


In [6]:
# Write CNN Code using inbuilt functions (Same as in Part-4)

class CNNModelPermutedMNIST(nn.Module):
    def __init__(self, kernel_sz=3, dropout_rate=0.25):
        super(CNNModelPermutedMNIST, self).__init__()  
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_sz, padding=(kernel_sz-1)//2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_sz, padding=(kernel_sz-1)//2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10) 

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
model = CNNModelPermutedMNIST()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    correct_val = 0
    total_val = 0
    val_loss = 0
    
    for images, labels in perm_train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted==labels).sum().item()

    for images, labels in perm_val_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        total_val += labels.size(0)
        _, predicted = outputs.max(1)
        correct_val += (predicted==labels).sum().item()

    print(f"Epoch {epoch + 1}, Train Loss: {train_loss / len(perm_train_loader)}, Train Accuracy: {100*correct_train/total_train:.2f}%")
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / len(perm_val_loader)}, Validation Accuracy: {100*correct_val/total_val:.2f}%")

Epoch 1, Train Loss: 0.6482178148031235, Train Accuracy: 83.31%
Epoch 1, Validation Loss: 0.30686817506447117, Validation Accuracy: 90.43%
Epoch 2, Train Loss: 0.2618828623890877, Train Accuracy: 91.69%
Epoch 2, Validation Loss: 0.2445481448592816, Validation Accuracy: 92.67%
Epoch 3, Train Loss: 0.19794271129121382, Train Accuracy: 93.72%
Epoch 3, Validation Loss: 0.25570842790159776, Validation Accuracy: 92.67%
Epoch 4, Train Loss: 0.1700604790871342, Train Accuracy: 94.64%
Epoch 4, Validation Loss: 0.25350740180410286, Validation Accuracy: 92.64%
Epoch 5, Train Loss: 0.14909495710084836, Train Accuracy: 95.30%
Epoch 5, Validation Loss: 0.22783119528197704, Validation Accuracy: 93.70%
Epoch 6, Train Loss: 0.12796348215763767, Train Accuracy: 95.87%
Epoch 6, Validation Loss: 0.21057355996379826, Validation Accuracy: 93.83%
Epoch 7, Train Loss: 0.11622141516705355, Train Accuracy: 96.23%
Epoch 7, Validation Loss: 0.21707254981602284, Validation Accuracy: 94.23%
Epoch 8, Train Loss: 0.1

In [7]:
learning_rates = [0.001, 0.008]
kernel_sizes = [3, 5]
dropout_rates = [0, 0.25]

for lr in learning_rates:
    for kernel_sz in kernel_sizes:
        for dropout_rate in dropout_rates:
            run = wandb.init(
                        project="SMAI_As3_Q5_4", entity="mitanshkayathwal"
                    )
            
            model = CNNModelPermutedMNIST(kernel_sz, dropout_rate)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr)
            num_epochs = 10
            for epoch in range(num_epochs):
                model.train()
                train_loss = 0
                correct_train = 0
                total_train = 0
                correct_val = 0
                total_val = 0
                val_loss = 0
                
                for images, labels in perm_train_loader:
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total_train += labels.size(0)
                    correct_train += (predicted==labels).sum().item()

                for images, labels in perm_val_loader:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    total_val += labels.size(0)
                    _, predicted = outputs.max(1)
                    correct_val += (predicted==labels).sum().item()

                train_loss /= len(perm_train_loader)
                val_loss /= len(perm_val_loader)

                train_accuracy = 100* correct_train/total_train
                val_accuracy = 100*correct_val/total_val
            wandb.log({"Learning Rate": lr,
                           "Kernel Size": kernel_sz,
                           "Dropout Rate": dropout_rate,
                           "Train Accuracy": train_accuracy,
                           "Validation Accuracy": val_accuracy,
                           "Train Loss": train_loss,
                           "Validation Loss": val_loss
                           })

wandb: Currently logged in as: mitanshkayathwal. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Dropout Rate,▁
Kernel Size,▁
Learning Rate,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Dropout Rate,0
Kernel Size,3
Learning Rate,0.001
Train Accuracy,98.31458


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Dropout Rate,▁
Kernel Size,▁
Learning Rate,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Dropout Rate,0.25
Kernel Size,3
Learning Rate,0.001
Train Accuracy,97.1375


...

Dropout Rate,▁
Kernel Size,▁
Learning Rate,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Dropout Rate,0
Kernel Size,3
Learning Rate,0.008
Train Accuracy,83.36875


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Dropout Rate,▁
Kernel Size,▁
Learning Rate,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Dropout Rate,0.25
Kernel Size,3
Learning Rate,0.008
Train Accuracy,71.61667


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Dropout Rate,▁
Kernel Size,▁
Learning Rate,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation Loss,▁
Dropout Rate,0
Kernel Size,5
Learning Rate,0.008
Train Accuracy,83.80208


# Best Hyperparameters found using WandB:

Kernel Size: 3

Learning Rate: 0.001

Training Accuracy: 97.14%

Validation Accuracy: 94.76%

Training Loss: 0.088

Validation Loss: 0.2081

In [8]:
model = CNNModelPermutedMNIST(3, 0.25)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    
    for images, labels in perm_train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [9]:
test_accuracy = 0
test_loss = 0
total_test = 0
correct_test = 0

for images, labels in perm_test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        total_test += labels.size(0)
        _, predicted = outputs.max(1)
        correct_test += (predicted==labels).sum().item()

                
test_loss /= len(perm_test_loader)
test_accuracy = 100* correct_test/total_test

print("Test loss: " + str(test_loss))
print("Test Accuracy: " + str(test_accuracy))

Test loss: 0.2059216628349416
Test Accuracy: 94.71


## Analysis:

a. Double MNIST:

MLP: MLPs are  well-suited for tabular data or simple patterns, but they do not capture spatial relationships efficiently in images. In this case MLP performs very poorly for DoubleMNIST as can be seen through the low accuracies while training and testing.

CNN: CNNs are designed to capture spatial features effectively. They perform significantly better on image datasets like MN2IST. The convolutional layers enable the CNN to detect hierarchical features, leading to better performance.

b. Permuted MNIST:

MLP: Since the spatial structure of images is destroyed in Permuted MNIST, MLPs face a challenging task of recognizing digits without relying on spatial relationships between pixels. MLPs perform better in this case

CNN: CNNs do not perform as well on Permuted MNIST because they are designed to exploit spatial structures. They might struggle to adapt to the new feature representations.

Observed Differences and Challenges:

Training and Evaluation Times: CNNs are generally more computationally intensive during training due to their convolutional layers. This results in longer training times compared to MLPs.

Overfitting: CNNs have more parameters and are prone to overfitting when there's limited data. Overfitting can be mitigated with techniques like dropout and weight decay. MLPs, having fewer parameters, might be less prone to overfitting.

Potential for Overfitting:

Overfitting in CNN: CNNs have a higher potential for overfitting, especially on smaller datasets, due to their larger number of learnable parameters. Regularization techniques like dropout and weight decay are essential to prevent overfitting.

Overfitting in MLP: While MLPs are less prone to overfitting due to fewer parameters, they can still overfit, particularly when the model is too complex or the dataset is small. Regularization is also useful for MLPs.